In [5]:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import pickle
import numpy as np
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
import tweepy
import regex as re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
import itertools



In [41]:
#Credentials 
consumer_key=''
consumer_secret=''
key=''
secret=''


In [132]:
#Scrape tweets using Twitter API
#to show example I have used fetched tweets for only 1 screen name @realself
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(key, secret)

api = tweepy.API(auth)
a=api.user_timeline(screen_name='realself',count=100)



a=api.user_timeline(screen_name='realself',count=100)
realself_tweets=[]
realself_tweets.extend(a)

file = open('tweet.json', 'w', encoding='utf8') 
for status in realself_tweets:
        json.dump(status._json,file,sort_keys = True,indent = 4)

In [133]:
#Tweets cleaner 
tok = WordPunctTokenizer()
#Patterns to remove links : @ , https, www
pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
#change the negative words
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

#Using BeautifulSoup to parse through the tweets text
def tweet_cleaner_updated(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    lower_case = stripped.lower()
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
#Remove the unnessesary white space 
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()

In [142]:
df = pd.read_csv("../version1tweets.csv",header=None,usecols=[0,5],names=['sentiment','text'],encoding='latin-1')
df['sentiment'] = df['sentiment'].map({0: 0, 4: 1})
df.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [146]:
%%time
print ("Cleaning the tweets...\n")
clean_tweet_texts = []
for i in range(0,len(df)):
    if( (i+1)%100000 == 0 ):
        print ("Tweets %d of %d has been processed" % ( i+1, len(df) ))                                                                    
    clean_tweet_texts.append(tweet_cleaner_updated(df['text'][i]))

Cleaning the tweets...

Tweets 100000 of 1600000 has been processed
Tweets 200000 of 1600000 has been processed
Tweets 300000 of 1600000 has been processed
Tweets 400000 of 1600000 has been processed
Tweets 500000 of 1600000 has been processed
Tweets 600000 of 1600000 has been processed
Tweets 700000 of 1600000 has been processed
Tweets 800000 of 1600000 has been processed
Tweets 900000 of 1600000 has been processed
Tweets 1000000 of 1600000 has been processed
Tweets 1100000 of 1600000 has been processed
Tweets 1200000 of 1600000 has been processed
Tweets 1300000 of 1600000 has been processed
Tweets 1400000 of 1600000 has been processed
Tweets 1500000 of 1600000 has been processed
Tweets 1600000 of 1600000 has been processed
CPU times: user 8min 21s, sys: 25.9 s, total: 8min 47s
Wall time: 8min 48s


In [147]:
# Saving the processed twets
clean_df = pd.DataFrame(clean_tweet_texts,columns=['text'])
clean_df['target'] = df.sentiment
clean_df.to_csv('clean_tweet.csv',encoding='utf-8')

### Query : To get all the tweets with hastag #Seatle and show counts of unique hastags in the fetched tweets

In [127]:
#fetch 100 the #Seattle
b=api.search(q='#Seattle',count=100)
#Using regular expression filter out all the hastags
all_tweets = [re.findall(r"#(\w+)", tweet.text) for tweet in b ]
all_tweets[:5]


[['babies',
  'BabyYodaProblems',
  'BabiesLivesMatter',
  'cutebaby',
  'CutenessOverload'],
 ['security', 'Seattle'],
 [],
 ['Seattle'],
 ['seattle', 'tacomawashington']]

In [130]:
#Convert all the hastags into lowercase
merged = list(itertools.chain.from_iterable(all_tweets))
merged1 =map(str.lower, merged)

In [131]:
#show the value count of all the distinct hastags
merged = pd.Series(merged1)
merged.value_counts()

seattle                     81
tacomawashington            10
corvid                       5
zerostatereflex              5
birds                        5
wabirds                      5
crow                         5
seattlestrong                2
seat                         2
tiroteo                      2
yalit                        2
washington                   2
cute                         2
bekindtooneanother           2
georgroth                    1
babieslivesmatter            1
cake                         1
whenitrainsitpours           1
babies                       1
jazzal                       1
madeinusa                    1
wa                           1
bradleytusk                  1
thursdayvibes                1
art                          1
onevoice1                    1
youthambassadors             1
guncontrolnow                1
hotels                       1
church                       1
                            ..
viru                         1
seattles